___
# Merge csv files for ETF shares, CME Open Interest, and CFTC COT reports.
___

In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import os, sys
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)

import zipfile
import urllib.request
from PIL import Image
import jupyter_utilities as ju
import importlib
import pandasql as psql

# Make important folders
TEMP_FOLDER = './temp_folder'
try:
    os.mkdir(TEMP_FOLDER)
except:
    pass
SAVE_IMAGE_FOLDER = f'{TEMP_FOLDER}/gold'
try:
    os.mkdir(SAVE_IMAGE_FOLDER)
except:
    pass

def to_int(s):
    try:
        return int(float(str(s)))
    except:
        print(f'to_int exception on value:{s}')
        return None

pd.set_option('display.max_colwidth',1000)
if os.path.abspath('../')  not in sys.path:
    if '.' not in sys.path:
        sys.path.append(os.path.abspath('../'))
import barchart_api as bcapi

    
cme_csv_save_folder = './cme_oi_data'
cot_data_path = './cot_net_new_history.csv'
etf_data_path = './etf_cap_hist.csv'

In [2]:
importlib.reload(ju)

<module 'jupyter_utilities' from '/Users/bperlman1/Documents/billybyte/pyliverisk/jupyter_notebooks/cme_open_interest/jupyter_utilities.py'>

### Define commodity and etf identifiers in the csv files

In [38]:
OI_ID_GOLD = 'GOLD FUTURES'
OI_ID_SILVER = 'SILVER FUTURES'
OI_ID_CL = 'CRUDE OIL LIGHT SWEET FUTURES'
OI_ID_NG = 'NATURAL GAS FUTURES'
OI_ID_10Y = '10Y NOTE FUTURE'
OI_ID_SPY = 'E-MINI S&P 500 FUTURE'
OI_ID_SOYB = 'SOYBEAN FUTURE'
OI_ID_CORN = 'CORN FUTURE'
OI_ID_WHEAT = 'CHICAGO SRW WHEAT FUTURE'
OI_ID_COTTON = 'COTTON FUTURES'
OI_ID_EURO = 'EURO FX FUTURE'
OI_ID_ED = 'EURODOLLAR FUTURE'
OI_ID_HG = 'HIGH GRADE COPPER FUTURES'

COT_ID_GOLD= 'GOLD - COMMODITY EXCHANGE INC.'
COT_ID_SILVER= 'SILVER - COMMODITY EXCHANGE INC.'
# COT_ID_CL = 'CRUDE OIL, LIGHT SWEET'
COT_ID_CL = 'CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE'
COT_ID_NG = 'NATURAL GAS - NEW YORK MERCANTILE EXCHANGE'
COT_ID_10Y = '10-YEAR U.S. TREASURY NOTES - CHICAGO BOARD OF TRADE'
COT_ID_SPY = 'E-MINI S&P 500 STOCK INDEX - CHICAGO MERCANTILE EXCHANGE'
COT_ID_SOYB = 'SOYBEANS - CHICAGO BOARD OF TRADE'
# COT_ID_WHEAT = 'WHEAT - CHICAGO BOARD OF TRADE'
COT_ID_COTTON = 'COTTON NO. 2 - ICE FUTURES U.S.'
COT_ID_WHEAT = 'WHEAT-SRW - CHICAGO BOARD OF TRADE'
COT_ID_CORN = 'CORN - CHICAGO BOARD OF TRADE'
COT_ID_EURO = 'EURO FX - CHICAGO MERCANTILE EXCHANGE'
COT_ID_ED = '3-MONTH EURODOLLARS - CHICAGO MERCANTILE EXCHANGE'
COT_ID_HG = 'COPPER-GRADE #1 - COMMODITY EXCHANGE INC.'

ETF_ID_GOLD = 'GLD'
ETF_ID_SILVER = 'SLV'
ETF_ID_CL = 'USO'
ETF_ID_NG = 'UNG'
ETF_ID_10Y = 'AGG'
ETF_ID_SPY = 'SPY'
ETF_ID_COTTON = 'BAL'
ETF_ID_SOYB = 'SOYB'
ETF_ID_WHEAT = 'WEAT'
ETF_ID_CORN = 'CORN'
ETF_ID_EURO = 'FXE'
ETF_ID_ED = 'BSV'
ETF_ID_HG = 'CPER'

ETF_SHARES_DIVISOR_GOLD = 1000
ETF_SHARES_DIVISOR_SILVER = 5000
ETF_SHARES_DIVISOR_10Y = 1000
ETF_SHARES_DIVISOR_SPY = 500
ETF_SHARES_DIVISOR_COTTON = 50000
ETF_SHARES_DIVISOR_SOYB = 5000
ETF_SHARES_DIVISOR_WHEAT = 5000
ETF_SHARES_DIVISOR_CORN = 5000
ETF_SHARES_DIVISOR_EURO = 1000
ETF_SHARES_DIVISOR_CL = 4000
ETF_SHARES_DIVISOR_NG = 1250
ETF_SHARES_DIVISOR_ED = 1000000
ETF_SHARES_DIVISOR_HG = 2500


ID_DICT = {
    'gold':{'OI':OI_ID_GOLD,'COT':COT_ID_GOLD,'ETF':ETF_ID_GOLD,'ETF_DIVISOR':ETF_SHARES_DIVISOR_GOLD},
    'silver':{'OI':OI_ID_SILVER,'COT':COT_ID_SILVER,'ETF':ETF_ID_SILVER,'ETF_DIVISOR':ETF_SHARES_DIVISOR_SILVER},
#     '10Y':{'OI':OI_ID_10Y,'COT':COT_ID_10Y,'ETF':ETF_ID_10Y,'ETF_DIVISOR':ETF_SHARES_DIVISOR_10Y},
#     'spy':{'OI':OI_ID_SPY,'COT':COT_ID_SPY,'ETF':ETF_ID_SPY,'ETF_DIVISOR':ETF_SHARES_DIVISOR_SPY},
    'cotton':{'OI':OI_ID_COTTON,'COT':COT_ID_COTTON,'ETF':ETF_ID_COTTON,'ETF_DIVISOR':ETF_SHARES_DIVISOR_COTTON},
    'soyb':{'OI':OI_ID_SOYB,'COT':COT_ID_SOYB,'ETF':ETF_ID_SOYB,'ETF_DIVISOR':ETF_SHARES_DIVISOR_SOYB},
    'wheat':{'OI':OI_ID_WHEAT,'COT':COT_ID_WHEAT,'ETF':ETF_ID_WHEAT,'ETF_DIVISOR':ETF_SHARES_DIVISOR_WHEAT},
    'corn':{'OI':OI_ID_CORN,'COT':COT_ID_CORN,'ETF':ETF_ID_CORN,'ETF_DIVISOR':ETF_SHARES_DIVISOR_CORN},
#     'euro':{'OI':OI_ID_EURO,'COT':COT_ID_EURO,'ETF':ETF_ID_EURO,'ETF_DIVISOR':ETF_SHARES_DIVISOR_EURO},
    'cl':{'OI':OI_ID_CL,'COT':COT_ID_CL,'ETF':ETF_ID_CL,'ETF_DIVISOR':ETF_SHARES_DIVISOR_CL},
    'ng':{'OI':OI_ID_NG,'COT':COT_ID_NG,'ETF':ETF_ID_NG,'ETF_DIVISOR':ETF_SHARES_DIVISOR_NG},
#     'ed':{'OI':OI_ID_ED,'COT':COT_ID_ED,'ETF':ETF_ID_ED,'ETF_DIVISOR':ETF_SHARES_DIVISOR_ED},
    'hg':{'OI':OI_ID_HG,'COT':COT_ID_HG,'ETF':ETF_ID_HG,'ETF_DIVISOR':ETF_SHARES_DIVISOR_HG},
}


___
## Define help access routines
___

In [39]:
# COT helpers
def df_cot_by_name(dict_id='cl',df_cot=None):
    dfc = df_cot2 if df_cot is None else df_cot
    cot_id = ID_DICT[dict_id]['COT']
    return dfc[dfc.Market_and_Exchange_Names==cot_id]


___
### Get cme open interest, COT and ETF data from csv files
___

In [40]:
# df_soyb = fetch_history(ETF_ID_SOYB,df_etf.date.min(),df_etf.date.max())
# print(df_etf.tail())
# print(df_soyb.tail())
# df_etf = df_etf.append(df_soyb,ignore_index=True)

In [41]:
df_oi = None
years = np.linspace(2013,2019,7,dtype=int)
df_oi = None
for y in years:
    df_temp = pd.read_csv(f'{cme_csv_save_folder}/cme_open_interest_{y}.csv')
    df_temp = df_temp[~df_temp.Open_Interest.isnull()]
    if df_oi is None:
        df_oi = df_temp.copy()
    else:
        df_oi = df_oi.append(df_temp,ignore_index=True)
        df_oi.index = list(range(len(df_oi)))
df_oi.Open_Interest = df_oi.Open_Interest.apply(to_int)
df_oi.Total_Volume = df_oi.Total_Volume.apply(to_int)
df_oi = df_oi[~df_oi.Total_Volume.isnull()]
print(f'oi length:{len(df_oi)}')
df_etf = pd.read_csv(etf_data_path)
df_etf['trade_date'] = df_etf.date.apply(ju.str_to_yyyymmdd)
print(f'etf length:{len(df_etf)}')
df_cot2 = pd.read_csv(cot_data_path)
df_cot2.As_of_Date_In_Form_YYMMDD = df_cot2.As_of_Date_In_Form_YYMMDD.apply(ju.str_to_date)
df_cot2.Market_and_Exchange_Names = df_cot2.Market_and_Exchange_Names.str.strip() 
print(f'cot length:{len(df_cot2)}')


/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (3,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.



to_int exception on value:**Total
Volume
to_int exception on value:ExPit
Volume
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on value:nan
to_int exception on 

### For some of the ETF's, get the data from yahoo, and ignore the shares data

In [42]:
def fetch_history(symbol,dt_beg,dt_end):
    df = pdr.DataReader(symbol, 'yahoo', dt_beg, dt_end)
    df['date'] = df.index
    df.date = df.date.apply(lambda d: str(d)[0:4] + "-" + str(d)[5:7] + "-" + str(d)[8:10])
    df['trade_date'] = df.date.apply(lambda d: int(str(d)[0:4] + str(d)[5:7] + str(d)[8:10]))
    df = df.sort_values('date')
    df.index = list(range(len(df)))
    # make adj close the close
    df['nav'] = df['Adj Close']
    df['symbol'] = symbol
    df['shares'] = 0
    df = df[['symbol','date','nav','shares','trade_date']]
    return df

    

### Find identifier strings for specific Open Interest and COT rows in their respective DataFrames
1. Enter values for oi_key_word, cot_key_word and etf_key_word below
2. Choose the product/market_and_exchange_name/symbol that has the highest open_interest or volume

In [43]:
df_cot2.columns.values


array(['Market_and_Exchange_Names', 'As_of_Date_In_Form_YYMMDD',
       'Open_Interest_All', 'Prod_Merc_Positions_Long_All',
       'Swap_Positions_Long_All', 'M_Money_Positions_Long_All',
       'Other_Rept_Positions_Long_All', 'NonRept_Positions_Long_All',
       'Tot_Rept_Positions_Long_All', 'Prod_Merc_Positions_Short_All',
       'Swap_Positions_Short_All', 'M_Money_Positions_Short_All',
       'Other_Rept_Positions_Short_All', 'NonRept_Positions_Short_All',
       'Tot_Rept_Positions_Short_All', 'prod_net', 'prod_ratio',
       'swap_net', 'swap_ratio', 'monman_net', 'monman_ratio',
       'other_net', 'other_ratio', 'nonrep_net', 'nonrep_ratio',
       'totrep_net', 'totrep_ratio'], dtype=object)

In [44]:
import pdb
def create_merged_df(commod_to_use):
    global df_oi,df_etf,df_cot2
    # step 1 set up ID's
    OI_ID = ID_DICT[commod_to_use]['OI']
    COT_ID = ID_DICT[commod_to_use]['COT']
    ETF_ID = ID_DICT[commod_to_use]['ETF']
    ETF_DIVISOR = ID_DICT[commod_to_use]['ETF_DIVISOR']
    #Step 2: make sure ID's produce only one contract
    oi_key_word = OI_ID.lower()
    cot_key_word = COT_ID.lower()
    etf_key_word = ETF_ID.lower()

    l = (list(filter(lambda s: oi_key_word == str(s).lower(),df_oi.Product_Description.unique())))
    df_oi_sub = df_oi[df_oi.Product_Description.isin(l)][['Product_Description','Open_Interest']]
    df_oi_gb = df_oi_sub.groupby('Product_Description',as_index=False).sum()

    l = (list(filter(lambda s: cot_key_word == str(s).lower(),df_cot2.Market_and_Exchange_Names.unique())))
    df_cot_single = df_cot2[df_cot2.Market_and_Exchange_Names==COT_ID]
    df_cot_sub = df_cot2[df_cot2.Market_and_Exchange_Names.isin(l)][['Market_and_Exchange_Names','Open_Interest_All']]
    df_cot_gb = df_cot_sub.groupby('Market_and_Exchange_Names',as_index=False).sum()

    l = (list(filter(lambda s: etf_key_word == str(s).lower(),df_etf.symbol.unique())))
    dtmin = str(df_cot_single.As_of_Date_In_Form_YYMMDD.min())[0:10]
    dtmax = str(datetime.datetime.now())[0:10]
    #pdb.set_trace()
    df_etf_single = fetch_history(ETF_ID,dtmin,dtmax)
    df_etf_gb = df_etf_single.groupby('symbol',as_index=False).sum()
    

    if len(df_oi_gb) + len(df_cot_gb) + len(df_etf_gb) == 3:
        print(f"all ID's for commodity: {commod_to_use} are OK")
    else:
        print(f"!!!!! ALL ID's FOR COMMODITY: {commod_to_use} ARE NOT OK!!!!!!")
        print(f'oi is OK: {len(df_oi_gb)==1}')
        print(f'cot is OK: {len(df_cot_gb)==1}')
        print(f'etf is OK: {len(df_etf_gb)==1}')
        raise ValueError(f'ambiguous ID name {OI_ID}')
    
    
    # Step 3: merge oi, cot and etf stuff
    df_oi_single = df_oi[df_oi.Product_Description == OI_ID][['trade_date','Open_Interest','Total_Volume']]
    df_oi_single.Open_Interest = df_oi_single.Open_Interest.apply(to_int)
    df_oi_single.Total_Volume = df_oi_single.Total_Volume.astype(float).astype(int)
    df_commod_net = df_cot_single.copy()

    df_commod_net['cot_yyyymmdd'] = df_commod_net.As_of_Date_In_Form_YYMMDD.apply(ju.str_to_yyyymmdd)
    df_commod_net = df_commod_net.sort_values('cot_yyyymmdd')
    df_commod_net.index = list(range(len(df_commod_net)))

    last_date = ju.str_to_date(str(df_commod_net.iloc[-1].cot_yyyymmdd),sep='') + datetime.timedelta(7)
    last_date_yyyymmdd = ju.str_to_yyyymmdd(last_date)
    df_commod_net['next_cot_yyyymmdd'] = list(df_commod_net[1:].cot_yyyymmdd) + [last_date_yyyymmdd]

    df_etf_oi = df_etf_single[['trade_date','nav','shares']].merge(df_oi_single,how='inner',on='trade_date')
    df_etf_oi['nav_diff'] = df_etf_oi.nav.pct_change()
    df_etf_oi['share_diff'] = df_etf_oi.shares.pct_change()
    df_etf_oi['oi_diff'] = df_etf_oi.Open_Interest.pct_change()
    q = f"select * from df_etf_oi inner join df_commod_net on df_etf_oi.trade_date >= df_commod_net.cot_yyyymmdd and df_etf_oi.trade_date < df_commod_net.next_cot_yyyymmdd"
    df_etf_oi_cot =  psql.sqldf(q, locals())
    
#     etf_oi_cols = list(df_etf_oi.columns.values)
#     cot_cols = list(cols_to_change.values()) + ['cot_yyyymmdd','next_cot_yyyymmdd']
#     etf_oi_cot_cols = etf_oi_cols + cot_cols
#     df_etf_oi_cot = df_etf_oi_cot[etf_oi_cot_cols]

    # step 4: create final,
    cot_cols = ['prod_net','monman_net','swap_net','other_net','nonrep_net','prod_ratio','monman_ratio','swap_ratio','other_ratio']
    df_final = df_etf_oi_cot[['trade_date','nav']+cot_cols][-1000:]
    df_final = df_final.loc[:,~df_final.columns.duplicated()]
    return df_final


### Create multi plot of all data commodities

In [45]:
ID_DICT.keys()

dict_keys(['gold', 'silver', 'cotton', 'soyb', 'wheat', 'corn', 'cl', 'ng', 'hg'])

In [46]:
import traceback
dict_df = {}
last_n_days = 1000
for k in ID_DICT.keys():
    print(f'processing {k}')
    try:
        df = create_merged_df(k)
        dict_df[k] = df.iloc[-last_n_days:]
    except Exception as e:
        print(f'EXCEPTION: {str(e)}')
        print(traceback.print_exc())

processing gold
all ID's for commodity: gold are OK
processing silver
all ID's for commodity: silver are OK
processing cotton
all ID's for commodity: cotton are OK
processing soyb
all ID's for commodity: soyb are OK
processing wheat
all ID's for commodity: wheat are OK
processing corn
all ID's for commodity: corn are OK
processing cl
all ID's for commodity: cl are OK
processing ng
all ID's for commodity: ng are OK
processing hg
all ID's for commodity: hg are OK


In [47]:
dict_df['cl'].columns.values

array(['trade_date', 'nav', 'prod_net', 'monman_net', 'swap_net',
       'other_net', 'nonrep_net', 'prod_ratio', 'monman_ratio',
       'swap_ratio', 'other_ratio'], dtype=object)

___
### Plot each df in dict_df using plotly
___

In [48]:
ddd = dict_df['cl']
net_cols = [c for c in ddd.columns.values if '_net' in c]
f = ju.plotly_plot(ddd[['trade_date','nav']+net_cols],x_column='trade_date',bar_plot=False,yaxis2_cols=['nav'])
iplot(f)

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:396: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [49]:
f = ju.plotly_plot(ddd[['trade_date','nav','monman_net','other_net']],x_column='trade_date',bar_plot=False,yaxis2_cols=['nav'])
iplot(f)

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:396: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [50]:
y_left_label = 'y nav'
y_right_label = 'monman_net'

df_all_in = None
for k in dict_df.keys():
    df_in = dict_df[k]
    if len(df_in)>1:
        df_in_with_commod = df_in.copy()
        df_in_with_commod['symbol'] = k
        if df_all_in is None:
            df_all_in = df_in_with_commod.copy()
        else:
            df_all_in = df_all_in.append(df_in_with_commod,ignore_index=True)
    last_date = df_in.trade_date.max()
    xc = 'trade_date'
    plot_title = f'{k} last trade date {last_date}'
    try:
        fig = ju.plotly_pandas(df_in[['trade_date', 'nav', y_right_label]],x_column=xc,plot_title=plot_title,y_left_label=y_left_label,y_right_label=y_right_label)
        iplot(fig)
    except Exception as e:
        print(f'{k} {str(e)}')
df_all_in.to_csv(f'{TEMP_FOLDER}/df_all_in.csv',index=False)


/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:390: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:487: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



In [16]:
# this is the old "non plotly" plotting routine
# ju.multi_df_plot(dict_df=dict_df,x_column='trade_date',num_of_x_ticks=40,save_file_prefix='cot_nav_plot',save_image_folder='./temp_folder/saved_images')    


___
### Try various strategies based on above charts
___

In [17]:
dict_df['cl'].columns.values

array(['trade_date', 'nav', 'prod_net', 'monman_net', 'swap_net',
       'other_net', 'nonrep_net', 'prod_ratio', 'monman_ratio',
       'swap_ratio', 'other_ratio'], dtype=object)

In [18]:
field_to_chart = 'nav'
field_to_chart2 = 'monman_net'
for sym in dict_df.keys():
    df_all_in = dict_df[sym].copy()    
    df_all_in[f'{field_to_chart2}_prev'] = df_all_in[field_to_chart2].shift(1)
    df_all_in['is_transition'] = df_all_in.apply(lambda r: 1 if r[field_to_chart2] / r[f'{field_to_chart2}_prev'] < 0 else 0,axis=1)
    df_all_in['trans_sign'] = df_all_in.apply(lambda r: 1 if r[field_to_chart2] - r[f'{field_to_chart2}_prev'] > 0 else -1,axis=1)
    df_all_in['transition'] = df_all_in.is_transition * df_all_in.trans_sign
    df_all_in2 = df_all_in[['trade_date',field_to_chart,'transition']]
    fig = ju.plotly_pandas(df_all_in2[['trade_date',field_to_chart,'transition']],x_column='trade_date',plot_title=sym)
    iplot(fig)
# len(df_all_in[df_all_in.tran_count==1])/len(df_all_in)

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:390: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:487: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



___
### The cells below help you find commodities in df_cot2 and df_oi
___

In [19]:
list(filter(lambda s: str(s)!='nan' and 'NATURAL GAS FUTURES' in s and 'FUTURE' in s,list(set(df_oi.Product_Description))))

['NATURAL GAS FUTURES']

In [20]:
list(filter(lambda s: str(s)!='nan' and 'NATURAL' in s ,list(set(df_cot2.Market_and_Exchange_Names))))

['NATURAL GAS ICE HENRY HUB - ICE FUTURES ENERGY DIV',
 'NATURAL GAS PENULTIMATE ICE - ICE FUTURES ENERGY DIV',
 'NORTHERN NATURAL GAS - VENTURA (BASIS) - ICE FUTURES ENERGY DIV',
 'NATURAL GAS PIPELINE-MID-CONTINENT POOL PIN (BASIS - ICE FUTURES ENERGY DIV',
 'NATURAL GAS HENRY LD1 FIXED - ICE FUTURES ENERGY DIV',
 'NATURAL GAS PIPELINE-TEXOK (BASIS) - ICE FUTURES ENERGY DIV',
 'NATURAL GAS PIPELINE, TEXOK (BASIS) - ICE FUTURES ENERGY DIV',
 'NATURAL GAS - NEW YORK MERCANTILE EXCHANGE',
 'NATURAL GAS INDEX: ALGONQUIN CITY GATES - ICE FUTURES ENERGY DIV',
 'NORTHERN NATURAL GAS - DEMARCATION POOL (BASIS) - ICE FUTURES ENERGY DIV',
 'NATURAL GAS PIPELINE-MID-CONTINENT POOL PIN (BASIS) - ICE FUTURES ENERGY DIV',
 'NATURAL GAS ICE HENRY HUB LD1  - ICE FUTURES ENERGY DIV']

In [36]:
print(df_cot2[df_cot2.Market_and_Exchange_Names=='COTTON NO. 2 - NEW YORK COTTON EXCHANGE'].Open_Interest_All.sum())
print(df_cot2[df_cot2.Market_and_Exchange_Names=='COTTON NO. 2 - ICE FUTURES U.S.'].Open_Interest_All.sum())


0
62063783


In [22]:
list(filter(lambda s: 'Comm' in s,df_cot2.columns.values))

[]

In [23]:
df_cot2_cl = df_cot2[df_cot2.Market_and_Exchange_Names=='CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE']
df_cot2_cl = df_cot2_cl.sort_values('As_of_Date_In_Form_YYMMDD')
df_cot2_cl.tail()

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,Prod_Merc_Positions_Long_All,Swap_Positions_Long_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,NonRept_Positions_Long_All,Tot_Rept_Positions_Long_All,Prod_Merc_Positions_Short_All,...,swap_net,swap_ratio,monman_net,monman_ratio,other_net,other_ratio,nonrep_net,nonrep_ratio,totrep_net,totrep_ratio
8982,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE",2019-05-28,2102854,430074,135433,257197,307236,79889,2022965,432243,...,-446908.0,0.232566,201307.0,4.601843,237631.0,4.413993,10139.0,1.145362,-10139.0,0.995013
8983,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE",2019-06-04,2078407,435545,132244,239064,299883,82351,1996056,424124,...,-422969.0,0.238186,170376.0,3.480433,229792.0,4.278481,11380.0,1.160347,-11380.0,0.994331
8984,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE",2019-06-11,2057046,446808,138697,214928,300529,81684,1975362,428434,...,-376993.0,0.268954,118401.0,2.226610,233254.0,4.467172,6964.0,1.093201,-6964.0,0.996487
8985,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE",2019-06-18,2003618,435179,140591,223751,278998,78668,1924950,423312,...,-374536.0,0.272925,152679.0,3.148230,210408.0,4.067619,-418.0,0.994715,418.0,1.000217
8986,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE",2019-06-25,2004961,449040,138944,215133,281808,76092,1928869,426663,...,-397285.0,0.259113,161211.0,3.989707,217592.0,4.388439,-3895.0,0.951305,3895.0,1.002023


## END

In [24]:
comod = 'cl'
fig = ju.plotly_pandas(dict_df[comod][['trade_date', 'monman_net', 'monman_ratio']],x_column='trade_date',plot_title=comod)
iplot(fig)


/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:390: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/Users/bperlman1/Virtualenvs3/pyliverisk/lib/python3.6/site-packages/plotly/matplotlylib/renderer.py:487: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



In [25]:
def get_cot2(id):
    basic_cols = ['Market_and_Exchange_Names','As_of_Date_In_Form_YYMMDD','Open_Interest_All']
    long_cols = ['M_Money_Positions_Long_All','Other_Rept_Positions_Long_All','Prod_Merc_Positions_Long_All',
                'NonRept_Positions_Long_All']
    short_cols = ['M_Money_Positions_Short_All','Other_Rept_Positions_Short_All','Prod_Merc_Positions_Short_All',
                'NonRept_Positions_Short_All']

    df_ret = df_cot2[df_cot2.Market_and_Exchange_Names==ID_DICT[id]['COT']]
    df_ret = df_ret[basic_cols + long_cols + short_cols]
    return df_ret
dfc = get_cot2('cl')

In [26]:
get_cot2('cl').columns.values

array(['Market_and_Exchange_Names', 'As_of_Date_In_Form_YYMMDD',
       'Open_Interest_All', 'M_Money_Positions_Long_All',
       'Other_Rept_Positions_Long_All', 'Prod_Merc_Positions_Long_All',
       'NonRept_Positions_Long_All', 'M_Money_Positions_Short_All',
       'Other_Rept_Positions_Short_All', 'Prod_Merc_Positions_Short_All',
       'NonRept_Positions_Short_All'], dtype=object)

In [27]:
dfc[dfc.As_of_Date_In_Form_YYMMDD=='2019-04-23']

,Market_and_Exchange_Names,As_of_Date_In_Form_YYMMDD,Open_Interest_All,M_Money_Positions_Long_All,Other_Rept_Positions_Long_All,Prod_Merc_Positions_Long_All,NonRept_Positions_Long_All,M_Money_Positions_Short_All,Other_Rept_Positions_Short_All,Prod_Merc_Positions_Short_All,NonRept_Positions_Short_All
8977,"CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE",2019-04-23,2139213,352453,292217,448757,96400,22057,75254,507806,73290


In [35]:
dfc = df_cot2[df_cot2.Market_and_Exchange_Names==ID_DICT['cl']['COT']]

In [29]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(dfc[dfc.As_of_Date_In_Form_YYMMDD=='2019-04-23'].iloc[0])


Market_and_Exchange_Names         CRUDE OIL, LIGHT SWEET - NEW YORK MERCANTILE EXCHANGE
As_of_Date_In_Form_YYMMDD                                           2019-04-23 00:00:00
Open_Interest_All                                                               2139213
Prod_Merc_Positions_Long_All                                                     448757
Swap_Positions_Long_All                                                          158645
M_Money_Positions_Long_All                                                       352453
Other_Rept_Positions_Long_All                                                    292217
NonRept_Positions_Long_All                                                        96400
Tot_Rept_Positions_Long_All                                                     2042813
Prod_Merc_Positions_Short_All                                                    507806
Swap_Positions_Short_All                                                         670065
M_Money_Positions_Short_All     

In [30]:
from collections import OrderedDict
dfc2 = dfc.to_dict()#('rows')
dfc3 = OrderedDict(sorted(dfc2.items(), key=lambda x: dfc.columns.get_loc(x[0])))

In [31]:
dfc.columns.get_loc(list(dfc2.items())[2][0])

2

In [32]:
pd.DataFrame(dfc.to_dict('rows')).columns.values

array(['As_of_Date_In_Form_YYMMDD', 'M_Money_Positions_Long_All',
       'M_Money_Positions_Short_All', 'Market_and_Exchange_Names',
       'NonRept_Positions_Long_All', 'NonRept_Positions_Short_All',
       'Open_Interest_All', 'Other_Rept_Positions_Long_All',
       'Other_Rept_Positions_Short_All', 'Prod_Merc_Positions_Long_All',
       'Prod_Merc_Positions_Short_All', 'Swap_Positions_Long_All',
       'Swap_Positions_Short_All', 'Tot_Rept_Positions_Long_All',
       'Tot_Rept_Positions_Short_All', 'monman_net', 'monman_ratio',
       'nonrep_net', 'nonrep_ratio', 'other_net', 'other_ratio',
       'prod_net', 'prod_ratio', 'swap_net', 'swap_ratio', 'totrep_net',
       'totrep_ratio'], dtype=object)

In [33]:
dfc.columns.values#.get_loc(key='Open_Interest_All')

array(['Market_and_Exchange_Names', 'As_of_Date_In_Form_YYMMDD',
       'Open_Interest_All', 'Prod_Merc_Positions_Long_All',
       'Swap_Positions_Long_All', 'M_Money_Positions_Long_All',
       'Other_Rept_Positions_Long_All', 'NonRept_Positions_Long_All',
       'Tot_Rept_Positions_Long_All', 'Prod_Merc_Positions_Short_All',
       'Swap_Positions_Short_All', 'M_Money_Positions_Short_All',
       'Other_Rept_Positions_Short_All', 'NonRept_Positions_Short_All',
       'Tot_Rept_Positions_Short_All', 'prod_net', 'prod_ratio',
       'swap_net', 'swap_ratio', 'monman_net', 'monman_ratio',
       'other_net', 'other_ratio', 'nonrep_net', 'nonrep_ratio',
       'totrep_net', 'totrep_ratio'], dtype=object)

In [34]:
pd.DataFrame(dfc2).columns.values == dfc.columns.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])